<a href="https://colab.research.google.com/github/yshun2272/stock-dashboard/blob/main/230817_stock_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash
!pip install dash-bootstrap-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 4.2 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from datetime import datetime, timedelta

# Define the ticker symbols
tickers = ['EVGO', 'CHPT', 'BLNK', 'OIH', 'CHX']

# Function to fetch stock price history
def get_stock_price(ticker):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=730)  # 2 years

    stock = yf.Ticker(ticker)
    history = stock.history(start=start_date, end=end_date)

    return history

# Fetch stock price history for each ticker
data = {ticker: get_stock_price(ticker) for ticker in tickers}

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div(
    [
        html.H1(children='Stock Performance Dashboard'),
        dcc.Tabs(
            id='ticker-tabs',
            value=tickers[0],
            children=[
                dcc.Tab(
                    label=ticker,
                    value=ticker
                )
                for ticker in tickers
            ]
        ),
        html.Div(id='graph-content')
    ]
)

# Callback to update the graph content based on the selected ticker
@app.callback(Output('graph-content', 'children'), [Input('ticker-tabs', 'value')])
def update_graph(active_tab):
    if not active_tab:
        return []

    price_data = data.get(active_tab)

    if price_data is None:
        return []

    fig = go.Figure()

    # Add black line for stock price history
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Close'], mode='lines', name=active_tab, line=dict(color='black')))

    # Fetch volume data for first and last data points
    first_volume = price_data['Volume'].iloc[0]
    last_volume = price_data['Volume'].iloc[-1]

    fig.update_layout(
        title=f'Stock Price - {active_tab}',
        xaxis={'title': 'Date'},
        yaxis={'title': 'Price'}
    )

    # Convert volume to millions to the second decimal point
    first_volume_millions = first_volume / 1000000
    last_volume_millions = last_volume / 1000000

    # Add text boxes for first and last data points
    first_date = price_data.index[0].strftime('%Y-%m-%d')
    first_price = price_data['Close'].iloc[0]
    last_date = price_data.index[-1].strftime('%Y-%m-%d')
    last_price = price_data['Close'].iloc[-1]

    fig.add_annotation(
        text=f'{first_date}<br>Price: {first_price:.2f}<br>Volume: {first_volume_millions:.2f}M',
        xref='paper', yref='paper',
        x=0, y=1,
        showarrow=False,
        font=dict(size=12),
        bgcolor='yellow',
        opacity=0.5
    )

    fig.add_annotation(
        text=f'{last_date}<br>Price: {last_price:.2f}<br>Volume: {last_volume_millions:.2f}M',
        xref='paper', yref='paper',
        x=1, y=1,
        showarrow=False,
        font=dict(size=12),
        bgcolor='yellow',
        opacity=0.5
    )

    # Add a trendline connecting the first and last data points
    fig.add_trace(go.Scatter(x=[first_date, last_date], y=[first_price, last_price],
                             mode='lines', name='Trendline', line=dict(color='green' if last_price > first_price else 'red', dash='dot')))

    # Add bar chart for daily volume on secondary axis
    fig.add_trace(go.Bar(x=price_data.index, y=price_data['Volume'], name='Volume', yaxis='y2'))

    fig.update_layout(
        yaxis2=dict(title='Volume', overlaying='y', side='right'),
        barmode='stack'
    )

    return dcc.Graph(figure=fig)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



<ipython-input-2-401f27f28763>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-401f27f28763>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


<IPython.core.display.Javascript object>

In [20]:
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Define the ticker symbols
tickers = ['EVGO', 'CHPT', 'BLNK', 'OIH', 'CHX']

# Function to fetch stock price history
def get_stock_price(ticker):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=730)  # 2 years

    stock = yf.Ticker(ticker)
    history = stock.history(start=start_date, end=end_date)

    return history

# Fetch stock price history for each ticker and store in the data dictionary
data = {ticker: get_stock_price(ticker) for ticker in tickers}

# Normalize close price data by dividing by the first close price
for ticker, price_data in data.items():
    first_close = price_data['Close'].iloc[0]
    price_data['Normalized_Close'] = (price_data['Close'] / first_close - 1) * 100

# Define the specific dates and labels for vertical lines
vertical_lines = [
    {'date': '2021-11-06', 'label': 'BIL'},
    {'date': '2022-08-16', 'label': 'IRA'}
]

# Create a single Plotly graph for visualization
fig = go.Figure()

for ticker, price_data in data.items():
    color = 'rgba(255, 0, 0, 0.2)' if ticker in ['EVGO', 'CHPT', 'BLNK'] else 'rgba(0, 255, 0, 0.2)'
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Normalized_Close'],
                             mode='lines', name=ticker, line=dict(color=color), showlegend=False))

    # Annotate the end of each line with the ticker label
    last_date = price_data.index[-1]
    last_value = price_data['Normalized_Close'].iloc[-1]
    fig.add_annotation(
        text=ticker,
        x=last_date, y=last_value,
        showarrow=False,
        font=dict(size=12),
        xshift=-10 if ticker in ['EVGO', 'CHPT', 'BLNK'] else 10,
        yshift=10,
        align='center'
    )

# Add vertical dotted lines and labels
for line in vertical_lines:
    fig.add_shape(
        type='line',
        x0=line['date'], x1=line['date'],
        y0=-100, y1=100,
        line=dict(color='black', width=1, dash='dot')
    )
    fig.add_annotation(
        text=line['label'],
        x=line['date'], y=100,
        showarrow=False,
        font=dict(size=12),
        xshift=10,
        yshift=-10,
        align='right'
    )

fig.update_layout(
        yaxis={'title': 'Normalized Close Price (%)', 'tickvals': [-100, 0, 100], 'range': [-100, 100],
        'showgrid': False}
)

# Show the graph
fig.show()

